**<h5>0) Downloading the libraries</h5>**<br>
Before using the program you need to download the library pandas using 'pip install pandas' and mlxtend using 'pip install mlxtend'

**<h5>1) Selecting a file and creating the data frame</h5>**<br>
The program gives the user a list of files to select from and prompts the user to enter the name of one. once the name is entered the program saves the filename and then attempts to open the file. If the file fails to open or isn't found then it prints an error and exits the program. If the file is found then it is formatted appropriately and organized. The file reading and Organization is all done through the pandas library reading the CSV data sheet. The panda reader creates a 10 column data frame, takes each item and puts it into its own column, and creates a row for each transaction.

In [56]:
import pandas as pan
print("Amazon\nBest buy\nKmart\nTarget\nHomegoods")
File_name = input("please enter the name of the database: ")
try:
    csvFile = pan.read_csv(File_name+'.csv',header=None, delimiter=',', engine='python', names=range(10))
except:
    print("ERROR 404: file not found")
    exit()
print(csvFile.head())

Amazon
Best buy
Kmart
Target
Homegoods
          0            1                 2                 3  \
0  Desk Top      Printer       Flash Drive  Microsoft Office   
1   Lab Top  Flash Drive  Microsoft Office      Lab Top Case   
2   Lab Top      Printer       Flash Drive  Microsoft Office   
3   Lab Top      Printer       Flash Drive        Anti-Virus   
4   Lab Top  Flash Drive      Lab Top Case        Anti-Virus   

                     4             5                    6     7     8     9  
0             Speakers    Anti-Virus                 None  None  None  None  
1           Anti-Virus          None                 None  None  None  None  
2           Anti-Virus  Lab Top Case  External Hard-Drive  None  None  None  
3  External Hard-Drive  Lab Top Case                 None  None  None  None  
4                 None          None                 None  None  None  None  


**<h5>2) Organizing the data frame</h5>**<br>
After the transactions are collected from the CSV file and stored into the data frame it is organized using the pandas library. From there each item is put into its own column, if it appears in a transaction its put down as a 1, else its a 0. this happens for all of the items in the datasheet. 

In [57]:
matrix = \
    pan.get_dummies(csvFile.unstack().dropna()).groupby(level=1).sum()
transactions_count,Item_count =matrix.shape
print(matrix.head())

   Anti-Virus  Desk Top  Digital Camera  External Hard-Drive  Flash Drive  \
0           1         1               0                    0            1   
1           1         0               0                    0            1   
2           1         0               0                    1            1   
3           1         0               0                    1            1   
4           1         0               0                    0            1   

   Lab Top  Lab Top Case  Microsoft Office  Printer  Speakers  
0        0             0                 1        1         1  
1        1             1                 1        0         0  
2        1             1                 1        1         0  
3        1             1                 0        1         0  
4        1             1                 0        0         0  


**<h5>3) Setting the support and confidence</h5>**<br>
The user is asked to input the support and confidence using a number between 1 and 100. The number is then turned into a percentage and later used to filter out numbers that are below the given percentage.

In [58]:
support = input("please enter the minimum support between 1 and 100: ")
conf = input("please enter the confidence as a number between 1 and 100: ")

**<h5>4) Brute force</h5>**<br>
To start off, the program starts a timer to see how long it takes to run the program. From there it iterates through every item in the item list and then iterates through every combination of items. Each item is converted to a string and then each combination is saved to a list along with the support for each item set. Next the program iterates through each item and calculates the confidence for the association rules. If there are more than one item in the item set it takes the current item that is being used in the association rule and removed them from the list and adds it to the beginning with an arrow. finally the item is appended to a list with the support and confidence for each item set

In [59]:
from itertools import combinations
import time
start_time = time.time()
largeItemsets = []
patterns = []
for items in range(1, Item_count+1):
    for itemset in combinations(matrix, items):
        I_support = (matrix[list(itemset)].all(axis=1).sum())
        sup = (matrix[list(itemset)].all(axis=1).sum())/20
        s = [str(x) for x in itemset]
        patterns.append([", ".join(s), sup])
        for item in s:
            item_con = matrix[item].value_counts()[1]
            confidence = (I_support / item_con)
            temp = s.copy()
            if(len(s) > 1):
                temp.remove(item)
                temp.insert(0,item+" =>")
            largeItemsets.append([", ".join(temp), sup, confidence])
for i in range (0,13):
    print("_____________________________")
    print (largeItemsets[i])
    print (patterns[i])



_____________________________
['Anti-Virus', np.float64(0.7), np.float64(1.0)]
['Anti-Virus', np.float64(0.7)]
_____________________________
['Desk Top', np.float64(0.3), np.float64(1.0)]
['Desk Top', np.float64(0.3)]
_____________________________
['Digital Camera', np.float64(0.45), np.float64(1.0)]
['Digital Camera', np.float64(0.45)]
_____________________________
['External Hard-Drive', np.float64(0.45), np.float64(1.0)]
['External Hard-Drive', np.float64(0.45)]
_____________________________
['Flash Drive', np.float64(0.65), np.float64(1.0)]
['Flash Drive', np.float64(0.65)]
_____________________________
['Lab Top', np.float64(0.6), np.float64(1.0)]
['Lab Top', np.float64(0.6)]
_____________________________
['Lab Top Case', np.float64(0.7), np.float64(1.0)]
['Lab Top Case', np.float64(0.7)]
_____________________________
['Microsoft Office', np.float64(0.55), np.float64(1.0)]
['Microsoft Office', np.float64(0.55)]
_____________________________
['Printer', np.float64(0.5), np.float64(

**<h5>5) Brute force</h5>**<br>
The program creates a data frame for each frequent item as well as the support using the list for frequent items generated earlier. Afterwards another data frame is created with a column for the item set, the support, and the confidence. using the other list of association rules that were generated. The items in the data frame is then compared the the input support and confidence with the results being saved to a variable. the frequent items data frame is then compared the the user support and saved to another variable.

In [60]:
freqPat = pan.DataFrame(patterns, columns=["Itemset", "Support"])
freqItemset = pan.DataFrame(largeItemsets, columns=["Itemset", "Support", "Confidence"])
sresults = freqItemset[(freqItemset.Support >= (int(support)/100)) & (freqItemset.Confidence >= (int(conf)/100))]
patresults = freqPat[(freqPat.Support >= (int(support)/100))]

**<h5>4) Printing</h5>**<br>
Both of the frequent items and the association rules  data frames are printed out after being converted to a string. The execution time is then printed out at the end.

In [61]:
print("------------------Frequent items------------------")
print(patresults.to_string())
print("---------------Association Rules---------------------")
print(sresults.to_string())
print("--- %s seconds ---" % (time.time() - start_time))

------------------Frequent items------------------
                          Itemset  Support
0                      Anti-Virus     0.70
4                     Flash Drive     0.65
5                         Lab Top     0.60
6                    Lab Top Case     0.70
7                Microsoft Office     0.55
8                         Printer     0.50
9                        Speakers     0.55
13        Anti-Virus, Flash Drive     0.50
14            Anti-Virus, Lab Top     0.50
15       Anti-Virus, Lab Top Case     0.60
42  Flash Drive, Microsoft Office     0.55
43           Flash Drive, Printer     0.50
45          Lab Top, Lab Top Case     0.50
---------------Association Rules---------------------
                             Itemset  Support  Confidence
0                         Anti-Virus     0.70    1.000000
4                        Flash Drive     0.65    1.000000
5                            Lab Top     0.60    1.000000
6                       Lab Top Case     0.70    1.000000
7  

**<h5>5) Comparison</h5>**<br>
the results of the Brute force Apriori program is then compared to the results of apriori library and the fpgrowth library. The time for them is then printed out as well.

In [62]:
from mlxtend.frequent_patterns import apriori, association_rules 
Apriori_time = time.time()
frq_items = apriori(matrix, min_support = (int(support)/100),use_colnames = True) 
rules = association_rules(frq_items, metric ="confidence", min_threshold = (int(conf)/100))
rules = rules.sort_values(['support','confidence'], ascending =[False, False])
print(rules[['antecedents','consequents','support','confidence']].to_string())
print("--- Apriori: %s seconds ---" % (time.time() - Apriori_time))

from mlxtend.frequent_patterns import fpgrowth
Fpgrowth_time = time.time()
test= fpgrowth(matrix, min_support=(int(support)/100), use_colnames=True)
rules = association_rules(test, metric ="confidence", min_threshold = (int(conf)/100))
rules = rules.sort_values(['support','confidence'], ascending =[False, False])
print(rules[['antecedents','consequents','support','confidence']].to_string())
print("--- FPgrowth: %s seconds ---" % (time.time() - Fpgrowth_time))

           antecedents         consequents  support  confidence
4       (Lab Top Case)        (Anti-Virus)     0.60    0.857143
5         (Anti-Virus)      (Lab Top Case)     0.60    0.857143
7   (Microsoft Office)       (Flash Drive)     0.55    1.000000
6        (Flash Drive)  (Microsoft Office)     0.55    0.846154
8            (Printer)       (Flash Drive)     0.50    1.000000
2            (Lab Top)        (Anti-Virus)     0.50    0.833333
11           (Lab Top)      (Lab Top Case)     0.50    0.833333
0        (Flash Drive)        (Anti-Virus)     0.50    0.769231
9        (Flash Drive)           (Printer)     0.50    0.769231
1         (Anti-Virus)       (Flash Drive)     0.50    0.714286
3         (Anti-Virus)           (Lab Top)     0.50    0.714286
10      (Lab Top Case)           (Lab Top)     0.50    0.714286
--- Apriori: 0.009776592254638672 seconds ---
           antecedents         consequents  support  confidence
0       (Lab Top Case)        (Anti-Virus)     0.60    0.8

C:\Users\knock\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\knock\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
